## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
%tensorflow_version 2.x

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input
 
from tensorflow.keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''

'''Resnet 50 架構'''
model = keras.applications.ResNet50(include_top=False, weights='imagenet',
                                    input_tensor=input_tensor,
                                    pooling=None, classes=10)
model.summary()


TensorFlow 2.x selected.
94773248/94765736 [==============================] - 8s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
_

## 添加層數

In [2]:
x = model.output

'''可以參考Cifar10實作章節'''
NUM_CLASSES = 10
x = GlobalAveragePooling2D()(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))


Model深度： 177


## 鎖定特定幾層不要更新權重

In [0]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot = OneHotEncoder()
y_train = one_hot.fit_transform(y_train).toarray()
y_test = one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## Training

In [6]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=32, epochs=100)

Train on 50000 samples
Epoch 1/100
50000/50000 [==============================] - 84s 2ms/sample - loss: 1.4626 - accuracy: 0.5728
Epoch 2/100
50000/50000 [==============================] - 74s 1ms/sample - loss: 1.1989 - accuracy: 0.6557
Epoch 3/100
50000/50000 [==============================] - 74s 1ms/sample - loss: 1.0333 - accuracy: 0.6961
Epoch 4/100
50000/50000 [==============================] - 74s 1ms/sample - loss: 0.8678 - accuracy: 0.7400
Epoch 5/100
50000/50000 [==============================] - 73s 1ms/sample - loss: 0.7659 - accuracy: 0.7643
Epoch 6/100
50000/50000 [==============================] - 74s 1ms/sample - loss: 0.6262 - accuracy: 0.7986
Epoch 7/100
50000/50000 [==============================] - 74s 1ms/sample - loss: 0.5206 - accuracy: 0.8331
Epoch 8/100
50000/50000 [==============================] - 74s 1ms/sample - loss: 0.4764 - accuracy: 0.8504
Epoch 9/100
50000/50000 [==============================] - 74s 1ms/sample - loss: 0.4021 - accuracy: 0.8705
Epoch